In [1]:
import pandas as pd
import polars as pl

import numpy as np


# Analisis de Cantidad de Traqueostomias por Servicio de Salud

- En este apartado se quiere ver la cantidad de traqueostomias que se hicieron por Servicio de Salud.
Este analisis se realiza por anio.

## Carga de Archivos de GRD

In [2]:
COLUMNAS_POLARS = {
    "COD_HOSPITAL": pl.Int32,
    "CIP_ENCRIPTADO": pl.Int32,
    "SEXO": pl.Categorical,
    "FECHA_NACIMIENTO": pl.Date,
    "ETNIA": pl.Categorical,
    "PROVINCIA": pl.Categorical,
    "COMUNA": pl.Categorical,
    "NACIONALIDAD": pl.Categorical,
    "PREVISION": pl.Categorical,
    "SERVICIO_SALUD": pl.Categorical,
    "TIPO_PROCEDENCIA": pl.Categorical,
    "TIPO_INGRESO": pl.Categorical,
    "ESPECIALIDAD_MEDICA": pl.Categorical,
    "TIPO_ACTIVIDAD": pl.Categorical,
    "FECHA_INGRESO": pl.Date,
    "SERVICIOINGRESO": pl.Categorical,
    "FECHATRASLADO1": pl.Date,
    "SERVICIOTRASLADO1": pl.Categorical,
    "FECHATRASLADO2": pl.Date,
    "SERVICIOTRASLADO2": pl.Categorical,
    "FECHATRASLADO3": pl.Date,
    "SERVICIOTRASLADO3": pl.Categorical,
    "FECHATRASLADO4": pl.Date,
    "SERVICIOTRASLADO4": pl.Categorical,
    "FECHATRASLADO5": pl.Date,
    "SERVICIOTRASLADO5": pl.Categorical,
    "FECHATRASLADO6": pl.Date,
    "SERVICIOTRASLADO6": pl.Categorical,
    "FECHATRASLADO7": pl.Date,
    "SERVICIOTRASLADO7": pl.Categorical,
    "FECHATRASLADO8": pl.Date,
    "SERVICIOTRASLADO8": pl.Categorical,
    "FECHATRASLADO9": pl.Date,
    "SERVICIOTRASLADO9": pl.Categorical,
    "FECHAALTA": pl.Date,
    "SERVICIOALTA": pl.Categorical,
    "TIPOALTA": pl.Categorical,
    "CONDICIONDEALTANEONATO1": pl.Categorical,
    "PESORN1": pl.Int16,
    "SEXORN1": pl.Categorical,
    "RN1ESTADO": pl.Int8,
    "CONDICIONDEALTANEONATO2": pl.Categorical,
    "PESORN2": pl.Int16,
    "SEXORN2": pl.Categorical,
    "RN2ESTADO": pl.Int8,
    "CONDICIONDEALTANEONATO3": pl.Categorical,
    "PESORN3": pl.Int16,
    "SEXORN3": pl.Categorical,
    "RN3ESTADO": pl.Int8,
    "CONDICIONDEALTANEONATO4": pl.Categorical,
    "PESORN4": pl.Int16,
    "SEXORN4": pl.Categorical,
    "RN4ESTADO": pl.Int8,
    "DIAGNOSTICO1": str,
    "DIAGNOSTICO2": str,
    "DIAGNOSTICO3": str,
    "DIAGNOSTICO4": str,
    "DIAGNOSTICO5": str,
    "DIAGNOSTICO6": str,
    "DIAGNOSTICO7": str,
    "DIAGNOSTICO8": str,
    "DIAGNOSTICO9": str,
    "DIAGNOSTICO10": str,
    "DIAGNOSTICO11": str,
    "DIAGNOSTICO12": str,
    "DIAGNOSTICO13": str,
    "DIAGNOSTICO14": str,
    "DIAGNOSTICO15": str,
    "DIAGNOSTICO16": str,
    "DIAGNOSTICO17": str,
    "DIAGNOSTICO18": str,
    "DIAGNOSTICO19": str,
    "DIAGNOSTICO20": str,
    "DIAGNOSTICO21": str,
    "DIAGNOSTICO22": str,
    "DIAGNOSTICO23": str,
    "DIAGNOSTICO24": str,
    "DIAGNOSTICO25": str,
    "DIAGNOSTICO26": str,
    "DIAGNOSTICO27": str,
    "DIAGNOSTICO28": str,
    "DIAGNOSTICO29": str,
    "DIAGNOSTICO30": str,
    "DIAGNOSTICO31": str,
    "DIAGNOSTICO32": str,
    "DIAGNOSTICO33": str,
    "DIAGNOSTICO34": str,
    "DIAGNOSTICO35": str,
    "PROCEDIMIENTO1": str,
    "PROCEDIMIENTO2": str,
    "PROCEDIMIENTO3": str,
    "PROCEDIMIENTO4": str,
    "PROCEDIMIENTO5": str,
    "PROCEDIMIENTO6": str,
    "PROCEDIMIENTO7": str,
    "PROCEDIMIENTO8": str,
    "PROCEDIMIENTO9": str,
    "PROCEDIMIENTO10": str,
    "PROCEDIMIENTO11": str,
    "PROCEDIMIENTO12": str,
    "PROCEDIMIENTO13": str,
    "PROCEDIMIENTO14": str,
    "PROCEDIMIENTO15": str,
    "PROCEDIMIENTO16": str,
    "PROCEDIMIENTO17": str,
    "PROCEDIMIENTO18": str,
    "PROCEDIMIENTO19": str,
    "PROCEDIMIENTO20": str,
    "PROCEDIMIENTO21": str,
    "PROCEDIMIENTO22": str,
    "PROCEDIMIENTO23": str,
    "PROCEDIMIENTO24": str,
    "PROCEDIMIENTO25": str,
    "PROCEDIMIENTO26": str,
    "PROCEDIMIENTO27": str,
    "PROCEDIMIENTO28": str,
    "PROCEDIMIENTO29": str,
    "PROCEDIMIENTO30": str,
    "MEDICOINTERV1_ENCRIPTADO": pl.Int32,
    "FECHAPROCEDIMIENTO1": pl.Date,
    "FECHAINTERV1": pl.Date,
    "ESPECIALIDADINTERVENCION": pl.Categorical,
    "MEDICOALTA_ENCRIPTADO": pl.Int32,
    "USOSPABELLON": str,
    "IR_29301_COD_GRD": pl.Int32,
    "IR_29301_PESO": str,
    "IR_29301_SEVERIDAD": pl.Int8,
    "IR_29301_MORTALIDAD": pl.Int8,
    "HOSPPROCEDENCIA": str,
}

VALORES_NULOS_COLUMNAS = {
    "CIP_ENCRIPTADO": "SIN INFORMACIÓN",
    "PROCEDIMIENTO1": "DESCONOCIDO",
    "PROCEDIMIENTO2": "DESCONOCIDO",
    "PROCEDIMIENTO3": "DESCONOCIDO",
    "PROCEDIMIENTO5": "DESCONOCIDO",
    "IR_29301_COD_GRD": "DESCONOCIDO",
    "IR_29301_PESO": "DESCONOCIDO",
    "IR_29301_SEVERIDAD": "DESCONOCIDO",
    "IR_29301_MORTALIDAD": "DESCONOCIDO",
}

In [3]:
with pl.StringCache():
    dfp = pl.scan_csv(
        "input/*.txt",
        separator="|",
        dtypes=COLUMNAS_POLARS,
        null_values=VALORES_NULOS_COLUMNAS,
    )

    peso_en_float = pl.col("IR_29301_PESO").str.replace(",", ".").cast(pl.Float32, strict=True)
    estancia = pl.col("FECHAALTA") - pl.col("FECHA_INGRESO")
    anio = pl.col("FECHAALTA").dt.year()

    dfp = dfp.with_columns(
        [
            peso_en_float.alias("IR_29301_PESO"),
            estancia.alias("ESTANCIA"),
            anio.alias("ANIO_EGRESO"),
        ]
    )
    dfp = dfp.collect()


In [4]:
df = dfp.to_pandas()

## Analisis de Traqueostomia

In [5]:
procedimientos = df.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"]
contiene_nuestros_proc = procedimientos.isin(["31.1", "31.29"]).sum(axis=1).astype(bool)
proc_nacionales = df[contiene_nuestros_proc]


In [6]:
metricas = (
    proc_nacionales.groupby(["ANIO_EGRESO", "SERVICIO_SALUD"])
    .agg(
        n_egresos=("COD_HOSPITAL", "count"),
        peso_medio=("IR_29301_PESO", "mean"),
        estancia_media=("ESTANCIA", "mean"),
    )
    .sort_values(["ANIO_EGRESO", "n_egresos"], ascending=False)
)

In [7]:
metricas.to_excel("resumen_traqueostomia_hist.xlsx")

# Analisis de Prestaciones por Diagnostico

- En este apartado se quiere ver la cantidad de prestaciones que se realizan por cada diagnostico
principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|

## Analisis Exploratorio

### Obtener conteo de diagnosticos en el Torax

In [8]:
torax = dfp.filter(pl.col("COD_HOSPITAL") == 112103)
torax.groupby(["DIAGNOSTICO1"]).agg(pl.count()).sort(by="count", descending=True)


DIAGNOSTICO1,count
str,u32
"""I25.1""",1366
"""Z51.1""",811
"""U07.1""",706
"""Z51.4""",595
"""Z13.6""",489
"""Z29.1""",394
"""C34.1""",367
"""I35.0""",365
"""Z45.0""",322


- De estos resultados, se puede ver que el diagnostico Z51.4 es el diagnostico mas frecuente
entre 2019 y 2021. Tuvo 1366 egresos.

- A modo de ejemplo, se quiere ver el dataframe completo del diagnostico Z51.4, y analizar
los procedimientos que tiene incluidos.

### Analisis de conteo de procedimientos

### Forma 1

In [9]:
from functools import reduce

i251 = torax.filter(pl.col("DIAGNOSTICO1") == "I25.1").to_pandas()
listas_procedimientos = (
    i251.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"].fillna("").agg("-".join, axis=1).str.split("-")
)


sumado = reduce(lambda x, y: x + y, listas_procedimientos.values)

### Forma 2

In [10]:
cambiada = pd.melt(
    i251, id_vars="DIAGNOSTICO1", value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)]
)
cambiada["value"].value_counts()


99.29    1196
99.19    1125
89.54    1035
93.96     930
37.22     806
         ... 
78.61       1
39.98       1
99.10       1
37.99       1
37.87       1
Name: value, Length: 257, dtype: int64

### Analisis Completo

- Se quiere ver la cantida dde procedimientos hechos por posicion de procedimiento. O sea,
en el "PROCEDIMIENTO1" que procedimientos se ingresaron, y cuales son sus frecuencias ordenadas
de mayor a menor?

- Tambien se quiere ver la frecuencia global. O sea, para el diagnostico X, cuales son los
procedimientos que mas se le realizan, independiente de la posicion del procedimiento.

In [58]:
import string

procedimientos = [f"PROCEDIMIENTO{i}" for i in range(1, 31)]
letras = list(string.ascii_uppercase) + ["a", "b", "c", "d"]
codificacion = dict(zip(procedimientos, letras))
codificacion_reversa = dict(zip(letras, procedimientos))

torax_pandas = torax.to_pandas()
long_prestaciones = pd.melt(
    torax_pandas,
    id_vars=["ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=procedimientos,
)

long_prestaciones["variable"] = long_prestaciones["variable"].replace(codificacion)

In [64]:
resumen_prestaciones_global = (
    long_prestaciones.groupby(["ANIO_EGRESO", "DIAGNOSTICO1"])["value"].value_counts()
    # .reset_index(name="count")
)

resumen_prestaciones_por_procedimiento = (
    long_prestaciones.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "variable"])["value"]
    .value_counts()
    .reset_index(name="count")
)

resumen_prestaciones_por_procedimiento["variable"] = resumen_prestaciones_por_procedimiento[
    "variable"
].replace(codificacion_reversa)
resumen_prestaciones_por_procedimiento = resumen_prestaciones_por_procedimiento.set_index(
    ["ANIO_EGRESO", "DIAGNOSTICO1", "variable", "value"]
)

In [65]:
resumen_prestaciones_global.to_excel("prestaciones_globales.xlsx")
resumen_prestaciones_por_procedimiento.to_excel("prestaciones_por_procedimiento.xlsx")